In [ ]:
!pip install mesa
import mesa
import numpy as np
from IPython.display import Javascript
import copy
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.1 MB/s eta 0:00:00


In [ ]:
def randomStrategy(hand, current_agent, other_agents):
    missing_cards = hand.missingCards(flat = True, exclude_unowned_groups = True)
    random_card = np.random.choice(missing_cards)
    other_agent = np.random.choice(other_agents)
    return random_card, other_agent

def ownerStrategy(hand, current_agent, other_agents):
    missing_cards = hand.missingCards(flat = True, exclude_unowned_groups = True)
    possible_facts = []
    print("While considering what card to ask for, I consider the following facts: ")
    for quartet_group in hand.quartet_groups:
        for knowledge in quartet_group.knowledge:
            if type(knowledge) == Fact and knowledge.agent != current_agent and knowledge.card in missing_cards and knowledge.owns == True:
                possible_facts.append(knowledge)

    if len(possible_facts) != 0:
        print(possible_facts)
        chosen_fact = np.random.choice(possible_facts)
        return chosen_fact.card, chosen_fact.agent

    for quartet_group in hand.quartet_groups:
        for knowledge in quartet_group.knowledge:
            if type(knowledge) == ExclusiveOr and knowledge.card in missing_cards:
                possible_facts.append(knowledge)

    if len(possible_facts) != 0:
        print(possible_facts)
        chosen_fact = np.random.choice(possible_facts)
        chosen_agent = np.random.choice(chosen_fact.other_agents)
        return chosen_fact.card, chosen_agent
    else:
        print("ERROR: no possible facts or exclusiveOrs found, knowledge: ")
        print(current_agent)
        print(hand.getCards())
        for quartet_group in hand.quartet_groups:
            print(quartet_group.knowledge)

In [ ]:
class Fact:
    def __init__(self, agent, card, owns = True):
        self.agent = agent
        self.card = card
        self.owns = owns

    def considerFact(self, fact):
        if self.card != fact.card:
            return True
        if self.agent == fact.agent:
            # New fact is either identical to me or an update.
            if self.owns == fact.owns:
                # Identical, so this fact is not contradictory
                return True
            else:
                # Ownership changed: current fact is irrelevant
                return False
        if self.owns and fact.owns:
            # The card has a new owner; I am irrelevant
            return False
        # At this point, the new fact cannot be contradictory to me, so I stay relevant
        return True

    def getDerivableFacts(self, other_agents):
        derivable_facts = []
        if not self.owns:
            return derivable_facts
        for other_agent in other_agents:
            derivable_facts.append(Fact(other_agent, self.card, owns = False))
        return derivable_facts

    def __eq__(self, other):
        if not isinstance(other, Fact):
            return False

        return self.agent == other.agent and self.card == other.card and self.owns == other.owns

    def __repr__(self):
        return "fact_" + str(self.agent) + "_" + str(self.card) + "_" + str(self.owns)

    def __hash__(self):
        return hash((self.agent, self.card, self.owns))

class Disjunction:
    def __init__(self, other_agent, cards):
        self.facts = [Fact(other_agent, card, owns = True) for card in cards]
        self.other_agent = other_agent

    def getCards(self):
        return [fact.card for fact in self.facts]

    # Returns whether this knowledge is still relevant
    def considerFact(self, new_fact):
        # If the fact makes one element of the disjunct impossible, delete that element from cards
        # If the fact satisfies one element from the disjunct, delete the disjunct
        fact_to_remove = None
        for disjunct_fact in self.facts:
            if disjunct_fact == new_fact:
                # New fact satisfies disjuction, so delete the disjunct
                return False
            if not disjunct_fact.considerFact(new_fact):
                # If new fact makes current fact impossible, remove this fact from the facts
                fact_to_remove = disjunct_fact

        if (fact_to_remove != None):
            self.facts.remove(fact_to_remove)
        return True

    def __eq__(self, other):
        if not isinstance(other, Disjunction):
            return False

        return set(self.facts) == set(other.facts)

    def __repr__(self):
        return "disjunct_" + str(self.facts)

class ExclusiveOr:
    def __init__(self, other_agents, card):
        self.other_agents = copy.copy(other_agents)
        self.card = card

    # Returns whether this knowledge is still relevant
    def considerFact(self, fact):
        if self.card != fact.card:
            return True
        if fact.owns:
            # delete the ExclusiveOr
            return False
        else:
            try:
                print("From xor", self, "Remove agent", fact.agent)
                self.other_agents.remove(fact.agent)
            except ValueError:
                pass
        return True

    def __eq__(self, other):
        if not isinstance(other, ExclusiveOr):
            return False

        return self.other_agents == other.other_agents and self.card == other.card

    def __repr__(self):
        return "exclusiveor_" + str(self.other_agents) + "_" + str(self.card)



In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]

class Hand:
    def __init__(self, model, agent, other_agents):
        self.model = model
        self.agent = agent
        self.other_agents = other_agents
        self.quartet_groups = [QuartetGroup(quartet_set.group_name, quartet_set.card_names, agent, other_agents)
                              for quartet_set in self.model.quartet_sets]

    def getCards(self, flat = False):
        cards = [quarted_group.getCards() for quarted_group in self.quartet_groups]
        if flat:
            cards = flatten(cards)
        return cards

    def addCard(self, new_card):
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == new_card.group_name:
                quartet_group.addCard(new_card)
                return quartet_group.isComplete()

    def missingCards(self, flat = False, exclude_unowned_groups = False):
        missing_cards = [group.missingCards() for group in self.quartet_groups]
        if exclude_unowned_groups:
            new_missing_cards = []
            for missing_cards_group in missing_cards:
                if len(missing_cards_group) != 4:
                    new_missing_cards.append(missing_cards_group)
            missing_cards = new_missing_cards

        if flat:
            missing_cards = flatten(missing_cards)
        return missing_cards

    def getGroup(self, group_name):
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == group_name:
                return quartet_group

    def getCard(self, asked_card):
        quartet_group = self.getGroup(asked_card.group_name)
        for card in quartet_group.cards:
            if card.card_name == asked_card.card_name:
                return card

    def giveCard(self, asked_card):
        card = self.getCard(asked_card)
        if card.owned:
            card.owned = False
            return True
        return False

    def updateAgentHasGroup(self, other_agent_id, group_name, excluded_card):
        quartet_group = self.getGroup(group_name)
        quartet_group.updateAgentHasGroup(other_agent_id, excluded_card)

    def updateAgentHasCard(self, agent_id, card):
        quartet_group = self.getGroup(card.group_name)
        quartet_group.updateAgentHasCard(agent_id, card)

    def updateAgentDoesNotHaveCard(self, agent_id, card):
        quartet_group = self.getGroup(card.group_name)
        quartet_group.updateAgentDoesNotHaveCard(agent_id, card)

class QuartetGroup:
    def __init__(self, group_name, card_names, agent = -1, other_agents = []):
        self.group_name = group_name
        self.card_names = card_names
        self.agent = agent
        self.other_agents = other_agents
        self.cards = [QuartetCard(group_name, card_name, self) for card_name in card_names]
        self.knowledge = []
        self.print_logic = True #Makes the run way faster if set to false, set to true if the logic is necessary to show

        for card in self.cards:
            card_knowledge = ExclusiveOr(other_agents, card)
            self.knowledge.append(card_knowledge)
            self.knowledge.append(Fact(self.agent, card, owns = False))
            # self.potential_owners.append(-1)

    def getCardObject(self, asked_card):
        for card in self.cards:
            if asked_card == card:
                return card

    def addCard(self, new_card):
        for card in self.cards:
            if new_card == card:
                card.owned = True
                new_fact = Fact(self.agent, card)
                self.addFact(new_fact)

    def missingCards(self):
        current_cards = self.getCards()
        return [x for x in self.cards if x not in current_cards]

    def getCards(self):
        return [x for x in self.cards if x.owned]

    def isComplete(self):
        return np.all([x.owned for x in self.cards])

    def addFact(self, fact):
        if fact in self.knowledge:
            return
        updated_knowledge = []
        # Update current knowledge base
        for knowledge in self.knowledge:
            keep_knowledge = knowledge.considerFact(fact)
            if keep_knowledge:
                updated_knowledge.append(knowledge)

        # Add the fact
        updated_knowledge.append(fact)
        self.knowledge = updated_knowledge
        self.collapseExclusiveOrs()
        self.collapseDisjunctions()

        # If we learned that an agent now owns a certain card, also add the facts that the other agents do not own the card
        all_other_agents = self.other_agents + [self.agent]
        all_other_agents.remove(fact.agent)
        derivable_facts = fact.getDerivableFacts(all_other_agents)
        for derivable_fact in derivable_facts:
            if self.print_logic:
                print("From fact ", fact, " I (agent ", self.agent, ") derived ", derivable_fact, sep = '')
            self.addFact(derivable_fact)


    def collapseExclusiveOrs(self):
        collapsing_ors = []
        for knowledge in self.knowledge:
            if type(knowledge) == ExclusiveOr:
                if (len(knowledge.other_agents)) == 1:

                    collapsing_ors.append(copy.copy(knowledge))
        new_facts = []
        for collapsing_or in collapsing_ors:
            new_facts.append(Fact(collapsing_or.other_agents[0], collapsing_or.card, owns = True))
            if self.print_logic:
                print("From collapsing or ", collapsing_or, " I (agent ", self.agent, ") got new fact ", new_facts[-1], sep = '')
            self.knowledge.remove(collapsing_or)
        for fact in new_facts:
            self.addFact(fact)

    def collapseDisjunctions(self):
        collapsing_disjunctions = []
        for knowledge in self.knowledge:
            if type(knowledge) == Disjunction:
                if (len(knowledge.facts)) == 1:
                    collapsing_disjunctions.append(copy.copy(knowledge))
        new_facts = []
        for collapsing_disjunction in collapsing_disjunctions:
            new_facts.append(collapsing_disjunction.facts[0])
            if self.print_logic:
                print("From collapsing disjunction ", collapsing_disjunction, " I (agent ", self.agent, ") got new fact ", new_facts[-1], sep = '')
            self.knowledge.remove(collapsing_disjunction)
        for fact in new_facts:
            self.addFact(fact)

    def updateAgentHasGroup(self, other_agent, excluded_card):
        if excluded_card != None:
            if self.print_logic:
                print("From ask PA I (agent ", self.agent, ") got fact ", Fact(other_agent, excluded_card, owns = False), sep = '')
            self.addFact(Fact(other_agent, excluded_card, owns = False))
        # If there is already a disjunction for this agent, no extra info is learned
        for knowledge in self.knowledge:
            if type(knowledge) == Disjunction:
                if knowledge.other_agent == other_agent:
                    return

        new_knowledge = Disjunction(other_agent, self.cards)
        for knowledge in self.knowledge:
            if type(knowledge) == Fact:
                new_knowledge.considerFact(knowledge)

        self.knowledge.append(new_knowledge)
        if self.print_logic:
            print("As agent", self.agent, "I have this knowledge:", self.knowledge)

    def updateAgentHasCard(self, agent_id, card):
        card_object = self.getCardObject(card)
        if self.print_logic:
          print("From the 'has card' PA I (agent ", self.agent, ") got fact ", Fact(agent_id, card_object, owns = True), sep = '')
        self.addFact(Fact(agent_id, card_object, owns = True))

    def updateAgentDoesNotHaveCard(self, agent_id, card):
        card_object = self.getCardObject(card)
        if self.print_logic:
            print("From the 'does not have card' PA I (agent ", self.agent, ") got fact ", Fact(agent_id, card_object, owns = False), sep = '')
        self.addFact(Fact(agent_id, card_object, owns = False))

    def __repr__(self):
        return str([card for card in self.cards])

class QuartetCard:
    def __init__(self, group_name, card_name, quartet_group, owned = False):
        self.group_name = group_name
        self.card_name = card_name
        self.quartet_group = quartet_group
        self.owned = owned

    def __eq__(self, other):
        if not isinstance(other, QuartetCard):
            return NotImplemented

        return self.group_name == other.group_name and self.card_name == other.card_name

    def __repr__(self):
        return self.group_name + "_" + self.card_name

    def __hash__(self):
        return hash((self.group_name, self.card_name))

class QuartetAgent(mesa.Agent):

    def __init__(self, unique_id, model, strategy):
        super().__init__(unique_id, model)
        self.other_agents = [i for i in range(self.model.num_agents) if i != self.unique_id]
        self.hand = Hand(model, unique_id, self.other_agents)
        self.score = 0
        self.strategy = strategy


    def step(self):
        if not self.model.running:
            return
        print("I am agent", self.unique_id)
        print("I currently have these cards:")
        print(self.hand.getCards())
        print("I currently do not have these cards:")
        print(self.hand.missingCards())

        missing_cards = self.hand.missingCards(flat = True, exclude_unowned_groups = True)
        if len(missing_cards) == 0:
            self.model.skipTurn(self.unique_id)
            return

        asked_card, other_agent = self.strategy(self.hand, self.unique_id, self.other_agents)

        got_card = self.model.askForCard(self, other_agent, asked_card)
        if got_card:
            self.addCard(asked_card)
            print("Now, I have these cards:")
            print(self.hand.getCards())
            print("Now I can go again!")
            self.step()

    def addCard(self, new_card):
        has_complete_set = self.hand.addCard(new_card)
        if has_complete_set:
            self.score += 1
            self.model.announceCompleteGroup(self.unique_id, new_card.group_name)

    def giveCard(self, card):
        return self.hand.giveCard(card)

    def updateAgentHasGroup(self, other_agent_id, group_name, excluded_card):
        self.hand.updateAgentHasGroup(other_agent_id, group_name, excluded_card)

    def updateAgentHasCard(self, agent_id, card):
        self.hand.updateAgentHasCard(agent_id, card)

    def updateAgentDoesNotHaveCard(self, asked_agent_id, card):
        self.hand.updateAgentDoesNotHaveCard(asked_agent_id, card)



class QuartetModel(mesa.Model):

    def __init__(self, N, strategies):
        self.running = True
        self.num_agents = N
        self.strategies = strategies
        self.schedule = mesa.time.BaseScheduler(self)
        self.quartet_sets = [QuartetGroup('a', ['a_1', 'a_2', 'a_3', 'a_4']),
                             QuartetGroup('b', ['b_1', 'b_2', 'b_3', 'b_4']),
                             QuartetGroup('c', ['c_1', 'c_2', 'c_3', 'c_4'])]
        self.completed_sets = 0
        self.agent_can_ask_owned_card = False
        self.print_announcements = True
        # Create agents
        for i in range(self.num_agents):
            a = QuartetAgent(i, self, self.strategies[i])
            self.schedule.add(a)

        all_cards = []
        for quartet_set in self.quartet_sets:
            for quartet_card in quartet_set.cards:
                all_cards.append(quartet_card)
        np.random.shuffle(all_cards)

        cards_per_agent = 4
        for i in range(cards_per_agent):
            for agent in self.schedule.agents:
                current_card = all_cards.pop()
                agent.addCard(current_card)

    def step(self):
        """Advance the model by one step."""
        for agent in self.schedule.agents:
            print(agent.unique_id)
            print(agent.hand.getCards())
            for quartet_group in agent.hand.quartet_groups:
                print(quartet_group.knowledge)
        self.schedule.step()

    def askForCard(self, asking_agent, asked_agent_id, card):
        asking_agent_id = asking_agent.unique_id
        if self.print_announcements:
            print("PA: Agent", asking_agent_id, "asks agent", asked_agent_id, "for card", card)
        asked_agent_object = self.schedule.agents[asked_agent_id]
        self.announceAgentHasGroup(asking_agent, card)
        has_card = asked_agent_object.giveCard(card)
        if has_card:
            if self.print_announcements:
                print("PA: Agent", asked_agent_id, "had the card and gave it to", asking_agent_id)
            self.announceAgentHasCard(asking_agent_id, card)
        else:
            if self.print_announcements:
                print("PA: Agent", asked_agent_id, "did not have the card.")
            self.announceAgentDoesNotHaveCard(asked_agent_id, card)
        print("")
        return has_card

    def announceAgentHasGroup(self, asking_agent, card):
        # The asking agent has complete knowledge over its cards, so we can skip this
        # This is always a successful announcement
        group_name = card.group_name
        other_agents = [agent for agent in self.schedule.agents if agent != asking_agent]
        excluded_card = None
        if not self.agent_can_ask_owned_card:
            excluded_card = card
        for agent in other_agents:
            agent.updateAgentHasGroup(asking_agent.unique_id, group_name, excluded_card)

    def announceAgentHasCard(self, asking_agent_id, card):
        other_agents = [agent for agent in self.schedule.agents if agent != asking_agent_id]
        for agent in other_agents:
            agent.updateAgentHasCard(asking_agent_id, card)

    def announceAgentDoesNotHaveCard(self, asked_agent_id, card):
        other_agents = [agent for agent in self.schedule.agents if agent != asked_agent_id]
        for agent in other_agents:
            agent.updateAgentDoesNotHaveCard(asked_agent_id, card)

    def announceCompleteGroup(self, agent, group_name):
        if self.print_announcements:
            print("PA: Agent", agent, "has completed the set", group_name)
        self.completed_sets += 1
        if len(self.quartet_sets) == self.completed_sets:
            self.finishGame()

    def skipTurn(self, agent):
        if self.print_announcements:
            print("PA: Agent", agent, "cannot ask for any card and skips this turn.")

    def finishGame(self):
        print("----------------------------------------------------------------------------")
        print("The game is finished! Here are the scores:")
        for agent in self.schedule.agents:
            print("Agent", agent.unique_id, "has score", agent.score)
        self.running = False

    def getScores(self):
        scores = []
        for agent in self.schedule.agents:
            scores.append(agent.score)
        return scores

In [ ]:
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 500})'''))

strategyNames = [randomStrategy, ownerStrategy]

strategies = [randomStrategy, randomStrategy, ownerStrategy]
num_agents = 3
num_runs = 10

total_scores = [0, 0, 0]
plot_scores = []
for i in range(num_runs):
    model = QuartetModel(num_agents, strategies)
    model.run_model()
    total_scores = np.add(total_scores, model.getScores())
    plot_scores.append(total_scores)

# Plot each agent's line
for agent in range(num_agents):
    line_data = [d[agent] for d in plot_scores]  # Extract data for the current agent
    x = np.arange(len(line_data))  # Generate x values
    y = line_data  # Use the line data as y values
    plt.plot(x, y, label=f'Agent {agent+1}')  # Plot the line

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Progression of points for each agent')
plt.legend()

plt.show()


print(total_scores)

<IPython.core.display.Javascript object>

From fact fact_0_c_c_4_True I (agent 0) derived fact_1_c_c_4_False
From fact fact_0_c_c_4_True I (agent 0) derived fact_2_c_c_4_False
From fact fact_1_b_b_3_True I (agent 1) derived fact_0_b_b_3_False
From fact fact_1_b_b_3_True I (agent 1) derived fact_2_b_b_3_False
From fact fact_2_c_c_3_True I (agent 2) derived fact_0_c_c_3_False
From fact fact_2_c_c_3_True I (agent 2) derived fact_1_c_c_3_False
From fact fact_0_a_a_1_True I (agent 0) derived fact_1_a_a_1_False
From fact fact_0_a_a_1_True I (agent 0) derived fact_2_a_a_1_False
From fact fact_1_b_b_2_True I (agent 1) derived fact_0_b_b_2_False
From fact fact_1_b_b_2_True I (agent 1) derived fact_2_b_b_2_False
From fact fact_2_a_a_2_True I (agent 2) derived fact_0_a_a_2_False
From fact fact_2_a_a_2_True I (agent 2) derived fact_1_a_a_2_False
From fact fact_0_b_b_4_True I (agent 0) derived fact_1_b_b_4_False
From fact fact_0_b_b_4_True I (agent 0) derived fact_2_b_b_4_False
From fact fact_1_c_c_1_True I (agent 1) derived fact_0_c_c_1_F

TypeError: ignored

In [ ]:
print(total_scores)

[ 9 12  9]
